**Imports and Setup. The initialization cell below should run automatically and print out "Done with initialization!"**

In [ ]:
from esper.prelude import *
from rekall.video_interval_collection import VideoIntervalCollection
from esper.rekall import intrvllists_to_result, add_intrvllists_to_result
from query.models import Video, LabeledInterval, Labeler
from IPython.display import display, clear_output
import ipywidgets as widgets
import datetime

try:
    VIDEO
except NameError:
    VIDEO = None
    ESPER_WIDGET = None
    GROUPS = None

def choose_random_video():
    global VIDEO
    VIDEO = Video.objects.filter(ignore_film=False).order_by('?')[0]
    
    print('Selected film {} ({})'.format(video.title, video.year))

def cache_labels(b):
    global ESPER_WIDGET, GROUPS
    GROUPS = ESPER_WIDGET.groups
    print("Saved {}".format(datetime.datetime.now()))
    
def elements_to_intrvllist(segmentlist):
    return [
        (int(seg['min_frame']), int(seg['max_frame']), 0)
        for seg in segmentlist['segments']
    ]
    
def label_video():
    global VIDEO, ESPER_WIDGET, GROUPS
    conversations_cached = []
    scenes_cached = []
    if GROUPS is not None and len(GROUPS) > 0 and len(GROUPS[0]['elements']) == 2:
        conversations_cached = elements_to_intrvllist(GROUPS[0]['elements'][0])
        scenes_cached = elements_to_intrvllist(GROUPS[0]['elements'][1])
    result = intrvllists_to_result(VideoIntervalCollection({VIDEO.id: conversations_cached}), color='red')
    add_intrvllists_to_result(result, VideoIntervalCollection({VIDEO.id: scenes_cached}), color='blue')
    
    save = widgets.Button(
        description='Save Progress',
        disabled=False,
        button_style='success',
        tooltip='Save Progress'
    )
    ESPER_WIDGET = esper_widget(result, jupyter_keybindings=True, timeline_annotation_keys={'w': 0, 'e': 1})
    display(ESPER_WIDGET)
    display(save)
    
def prepare_orm_objects():
    global GROUPS
    conversation_labeler_name = 'conversations-manual-{}'.format(
        datetime.datetime.now().strftime('%Y-%m-%d-%H:%M:%S')
    )
    scene_labeler_name = 'scenes-manual-{}'.format(
        datetime.datetime.now().strftime('%Y-%m-%d-%H:%M:%S')
    )
    new_conversation_labeler, _ = Labeler.objects.get_or_create(name=conversation_labeler_name)
    new_scene_labeler, _ = Labeler.objects.get_or_create(name=scene_labeler_name)
    if GROUPS is not None and len(GROUPS) > 0 and len(GROUPS[0]['elements']) == 2:
        conversations = [
            LabeledInterval(
                min_frame = int(seg['min_frame']),
                max_frame = int(seg['max_frame']),
                video_id=VIDEO.id,
                labeler=new_conversation_labeler
            )
            for seg in GROUPS[0]['elements'][0]['segments']
        ]
        scenes = [
            LabeledInterval(
                min_frame = int(seg['min_frame']),
                max_frame = int(seg['max_frame']),
                video_id=VIDEO.id,
                labeler=new_conversation_labeler
            )
            for seg in GROUPS[0]['elements'][1]['segments']
        ]
        return new_conversation_labeler, new_scene_labeler, conversations + scenes
    else:
        return []

def save_orm_objects(intervals):
    LabeledInterval.objects.bulk_create(intervals)

def reset_notebook_state():
    global VIDEO, ESPER_WIDGET, GROUPS
    VIDEO = None
    ESPER_WIDGET = None
    GROUPS = None

print('Done with initialization!')

Step 1: Run the cell below to pick a random film to label conversations and scenes for.

In [ ]:
choose_random_video()

Step 2: Run the cell below to show the labeling interface.

Hover over the video and use `=` (on the keyboard) to expand the frame. Use `Shift-P` to start playing the video.

Use the `+` and `-` buttons to the right of the timeline to zoom in or out, and use the `<` and `>` buttons to the right of the timeline to scroll the timeline forward or backwards.

Use `w` to mark conversations and `e` to mark scenes. The first time you press `w` or `e`, the timeline will place a marker as the start of a conversation or scene. Use `w` or `e` again to mark the end of the conversation or scene. You can use `z` on the keyboard to undo your last action.

Do not mark opening logos as scenes.

Use `+` and `-` on your keyboard to speed up or slow down the video, and ``` on the keyboard to return the video back to normal speed.

Press the **Save Progress** button at any time to locally cache your labels (these will be persistent on page refreshes but not on kernel death).

In [ ]:
label_video()

Step 3: Run the two cells below to commit your labels to the database!

In [ ]:
conversation_labeler, scene_labeler, intervals = prepare_orm_objects()

In [ ]:
save_orm_objects(intervals)

Step 4: Run the cell below to reset your notebook state and start over from step 1 with a new film!

In [ ]:
reset_notebook_state()